<a href="https://colab.research.google.com/github/a36600248-eng/toy/blob/main/kohya_ss_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ====== 官方 kohya-ss GUI（稳定版）Colab 一键脚本 ======
%cd /content

from google.colab import drive
drive.mount('/content/drive')

# 1) 系统依赖
!apt-get -y update
!apt-get -y install git aria2

# 2) 建 venv：关键点是 --system-site-packages
#    这样 venv 里能直接用 Colab 预装的 torch/cuda（避免你之前那种 python 路径错乱）
!python3 -m venv --system-site-packages /content/kohya_venv
KPY="/content/kohya_venv/bin/python"
KPIP="/content/kohya_venv/bin/pip"

# 3) 升级 pip 基础工具
!{KPIP} -q install -U pip setuptools wheel

# 4) 拉官方 kohya_ss（GUI 版）
%cd /content
!rm -rf /content/kohya_ss
!git clone https://github.com/kohya-ss/kohya_ss.git
%cd /content/kohya_ss

# 5) 安装依赖：过滤掉 bitsandbytes（你现在的 CUDA 12.6 会把 bnb 搞炸）
#    依赖文件名可能是 requirements.txt / requirements_gui.txt / requirements_linux.txt 等
#    这里用“存在就装”的方式，尽量兼容不同版本仓库结构
!bash -lc 'set -e; \
  REQ=""; \
  if [ -f requirements.txt ]; then REQ="requirements.txt"; fi; \
  if [ -f requirements_gui.txt ]; then REQ="requirements_gui.txt"; fi; \
  if [ -f requirements_linux.txt ]; then REQ="requirements_linux.txt"; fi; \
  if [ -z "$REQ" ]; then echo "No requirements file found!"; ls -la; exit 1; fi; \
  echo "Using requirements file: $REQ"; \
  sed "/bitsandbytes/d" "$REQ" > /content/req_nobnb.txt; \
  /content/kohya_venv/bin/pip -q install -r /content/req_nobnb.txt; \
  /content/kohya_venv/bin/pip -q install -U voluptuous'

# 6) 启动 GUI（网页界面）
#    看到 “Running on public URL:” 那行就是你的访问链接
!{KPY} kohya_gui.py --share --headless


/content
Mounted at /content/drive
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,302 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [354 B]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,870 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,600 kB]
Get:13 https://ppa.laun

In [1]:
!wget "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3B4_orangemixs.safetensors" -O /content/model.safetensors

--2026-01-19 01:18:02--  https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3B4_orangemixs.safetensors
Resolving huggingface.co (huggingface.co)... 108.138.246.85, 108.138.246.67, 108.138.246.79, ...
Connecting to huggingface.co (huggingface.co)|108.138.246.85|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/638cac3a61eb5101751a23c4/21501367dc185b5e4a7f08084d5c8b8ef456dbb130c1e61aeb3e2eb9d1347967?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27AOM3B4_orangemixs.safetensors%3B+filename%3D%22AOM3B4_orangemixs.safetensors%22%3B&Expires=1768789082&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY4Nzg5MDgyfX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjM4Y2FjM2E2MWViNTEwMTc1MWEyM2M0LzIxNTAxMzY3ZGMxODViNWU0YTdmMDgwODRkNWM4YjhlZjQ1NmRiYjEzMGMxZTYxYWViM2UyZWI5ZDEzNDc5NjdcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSo

In [ ]:
# @title 一键预处理：自动重命名 + 智能打标 (GLM-4.6v)
# @markdown ### 1. 核心配置
import os
import glob
import base64
import time
import shutil

# ----------------------------------------------------------------------
# [必须填] 你的智谱AI API Key
API_KEY = "这里填入你的key_不要删掉双引号"

# [必须填] 你的图片文件夹路径
# 例如: /content/drive/MyDrive/Lora_Training/img/20_miku
IMAGE_FOLDER_PATH = "/content/drive/MyDrive/你的文件夹路径"

# [必须填] 这个角色是谁？(用于辅助AI识别特征)
# 例如: Hatsune Miku
CHARACTER_NAME = "填写角色英文名"

# [可选] 出自哪部作品？(如果不知道可以留空)
# 例如: Vocaloid
SERIES_NAME = "填写作品英文名"

# [可选] 触发词 (会自动加在标签最前面)
# 通常和角色名一致，例如: miku
TRIGGER_WORD = "miku"
# ----------------------------------------------------------------------

# 1. 安装 zai-sdk
print("🔧 正在检查环境...")
try:
    from zai import ZhipuAiClient
except ImportError:
    print("⬇️ 正在安装 zai-sdk...")
    !pip install zai-sdk -q
    from zai import ZhipuAiClient
    print("✅ 安装完成")

def rename_images_sequentially(folder_path):
    """将文件夹内的图片重命名为 001.ext, 002.ext ..."""
    print(f"🔄 正在检查文件名规范性...")

    extensions = ['*.png', '*.jpg', '*.jpeg', '*.webp', '*.bmp']
    files = []
    for ext in extensions:
        files.extend(glob.glob(os.path.join(folder_path, ext)))

    # 按原文件名排序，保证顺序一致
    files.sort()

    count = 0
    for i, old_path in enumerate(files):
        folder = os.path.dirname(old_path)
        ext = os.path.splitext(old_path)[1]

        # 目标新名字: 001.png, 002.jpg ...
        new_name = f"{i+1:03d}{ext}"
        new_path = os.path.join(folder, new_name)

        # 如果名字已经是 001.png 这种格式，就不动它
        if old_path == new_path:
            continue

        # 防止重名冲突，先重命名为一个临时名字
        temp_path = os.path.join(folder, f"temp_{int(time.time())}_{i}{ext}")
        os.rename(old_path, temp_path)
        # 再重命名为目标名字
        os.rename(temp_path, new_path)
        count += 1

    if count > 0:
        print(f"✅ 已将 {count} 张图片重命名为 001, 002... 格式")
    else:
        print(f"✅ 文件名已符合规范，无需重命名")

def generate_tags(client, image_path, char_name, series_name):
    """发送给 GLM-4.6v 获取标签"""

    with open(image_path, "rb") as img_file:
        img_base = base64.b64encode(img_file.read()).decode("utf-8")

    # 构建更有针对性的 Prompt
    series_info = f"from the series '{series_name}'" if series_name else ""

    prompt_text = f"""
    This image contains the character "{char_name}" {series_info}.
    I need to train a LoRA model to mimic this character's style and appearance.

    Please describe this image using Stable Diffusion tag format.
    Rules:
    1. Output strictly comma-separated English keywords.
    2. IMPORTANT: Describe specific traits of "{char_name}" (e.g., specific hair ornaments, eye shape, outfit details).
    3. Describe the pose, background, and framing.
    4. NO sentences. NO "The image shows...". Just tags.
    """

    try:
        response = client.chat.completions.create(
            model="glm-4.6v",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "image_url", "image_url": {"url": img_base}},
                        {"type": "text", "text": prompt_text}
                    ]
                }
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"   [Error] API调用失败: {e}")
        return None

def main():
    if "这里填入" in API_KEY:
        print("❌ 错误：请先在代码最上方填入你的 API Key！")
        return

    if not os.path.exists(IMAGE_FOLDER_PATH):
        print(f"❌ 错误：找不到路径 -> {IMAGE_FOLDER_PATH}")
        return

    # 1. 先执行重命名
    rename_images_sequentially(IMAGE_FOLDER_PATH)

    # 2. 初始化客户端
    client = ZhipuAiClient(api_key=API_KEY)

    # 3. 重新扫描（因为文件名变了）
    extensions = ['*.png', '*.jpg', '*.jpeg', '*.webp', '*.bmp']
    image_files = []
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(IMAGE_FOLDER_PATH, ext)))
    image_files.sort()

    total = len(image_files)
    print(f"\n🚀 开始智能打标 (角色: {CHARACTER_NAME})...")

    for i, img_path in enumerate(image_files):
        filename = os.path.basename(img_path)
        txt_path = os.path.splitext(img_path)[0] + ".txt"

        print(f"[{i+1}/{total}] 处理: {filename} ...", end="", flush=True)

        # 跳过已存在的txt
        if os.path.exists(txt_path):
            print(" ⏭️ 跳过")
            continue

        tags = generate_tags(client, img_path, CHARACTER_NAME, SERIES_NAME)

        if tags:
            # 清洗数据
            tags = tags.replace("Tags:", "").replace("Keywords:", "").strip()
            if tags.endswith("."): tags = tags[:-1]

            # 组合最终内容: 触发词 + 识别到的Tag
            final_content = tags
            if TRIGGER_WORD:
                final_content = f"{TRIGGER_WORD}, {tags}"

            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(final_content)
            print(" ✅")
            time.sleep(0.5)
        else:
            print(" ❌")

    print("\n🎉 全部处理完成！文件已重命名并生成标签。")

if __name__ == "__main__":
    main()

In [ ]:
# @title 😶‍🌫️ 终极版：右下角局部模糊 (消除水印痕迹)
import os
import glob
from PIL import Image, ImageFilter

# ================= 配置区域 =================
# [必须填] 你的图片文件夹路径
IMAGE_FOLDER_PATH = "/content/drive/MyDrive/你的文件夹路径"

# [重要] 模糊区域的大小 (像素)
# 只要能覆盖住水印即可
BLUR_WIDTH = 220
BLUR_HEIGHT = 80

# 模糊强度 (半径)
# 15-20 左右通常足以把字彻底糊成一团颜色
BLUR_RADIUS = 20
# ===========================================

def blur_watermark(img_path):
    """在图片右下角进行高斯模糊"""
    try:
        # 打开图片
        img = Image.open(img_path).convert('RGBA')
        w, h = img.size

        # 检查图片大小
        if h <= BLUR_HEIGHT or w <= BLUR_WIDTH:
            print(f" ⚠️ 图片太小，跳过: {os.path.basename(img_path)}")
            return False

        # 1. 切出右下角那一小块
        box = (w - BLUR_WIDTH, h - BLUR_HEIGHT, w, h)
        region = img.crop(box)

        # 2. 对这一小块进行强力模糊
        # 重复模糊几次确保文字彻底消失
        for _ in range(3):
            region = region.filter(ImageFilter.GaussianBlur(radius=BLUR_RADIUS))

        # 3. 贴回去
        img.paste(region, box)

        # 4. 保存
        if img_path.lower().endswith(('.jpg', '.jpeg')):
            img = img.convert('RGB')

        img.save(img_path, quality=100, subsampling=0)
        return True
    except Exception as e:
        print(f" ❌ 处理出错: {os.path.basename(img_path)} - {e}")
        return False

def main():
    if not os.path.exists(IMAGE_FOLDER_PATH):
        print(f"❌ 找不到路径: {IMAGE_FOLDER_PATH}")
        return

    extensions = ['*.png', '*.jpg', '*.jpeg', '*.webp', '*.bmp']
    image_files = []
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(IMAGE_FOLDER_PATH, ext)))
        image_files.extend(glob.glob(os.path.join(IMAGE_FOLDER_PATH, ext.upper())))

    image_files = sorted(list(set(image_files)))
    total = len(image_files)

    if total == 0:
        print("📂 没找到图片。")
        return

    print(f"📂 准备模糊处理 {total} 张图片...")
    print(f"😶‍🌫️ 模糊区域: 右下角 {BLUR_WIDTH}x{BLUR_HEIGHT}")

    success_count = 0
    for i, img_path in enumerate(image_files):
        filename = os.path.basename(img_path)
        print(f"[{i+1}/{total}] 处理: {filename}", end="", flush=True)

        if blur_watermark(img_path):
            print(" -> ✅ 已模糊")
            success_count += 1
        else:
            print("")

    print(f"\n🎉 全部完成！水印已变成一团这种颜色的雾气，AI 很难学到形状。")

if __name__ == "__main__":
    main()